<a href="https://colab.research.google.com/github/JozefSL/pyNotes/blob/main/EIA/NGwklyStorageInventoryAndChange.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import requests

In [2]:
from google.colab import userdata
api_key = userdata.get('api_key')

In [3]:
url = 'https://api.eia.gov/v2/natural-gas/stor/wkly/data/?api_key='+ api_key + '&frequency=weekly&data[0]=value&start=2019-01-01&sort[0][column]=period&sort[0][direction]=asc&facets[series][]='

In [4]:
response = requests.get(url + 'NW2_EPG0_SWO_R48_BCF')

In [5]:
df = pd.DataFrame(response.json().get('response').get('data'))[['period','value']]
df.columns = ['Period', 'value']
df['Period'] = df['Period'].apply(pd.to_datetime)
df.value = (df.value.astype('Int64')/1).astype('Int64')
df['year'] = df['Period'].dt.year
df['week'] = df['Period'].dt.isocalendar().week
df['change'] = df['value'] - df['value'].shift(1)

df.tail(3)

,Period,value,year,week,change
297,2024-09-13,3445,2024,37,58
298,2024-09-20,3492,2024,38,47
299,2024-09-27,3547,2024,39,55


In [6]:
ty = df.year.iloc[-1]
ly = ty-1

In [15]:
dfA = df.pivot(index='week', columns='year', values='value').astype('Int64') #.reset_index(drop='true')
dfA = dfA.drop(dfA.index[-1]).reset_index()  #eliminate the add years with 53 weeks

In [16]:
dfA.tail(3)

year,week,2019,2020,2021,2022,2023,2024
49,50,3411,3726,3362,3325,3577,<NA>
50,51,3250,3574,3226,3112,3490,<NA>
51,52,3192,3460,3195,2891,3476,<NA>


In [17]:
dfAA = pd.DataFrame(dfA.iloc[:, -6:-1].mean(axis=1))
dfAA.columns = ['5yAvg']
dfAA['5yMin'] = dfA.iloc[:, -6:-1].min(axis=1) # Actual minimum, also used to calculate stack area bottom
dfAA['5yMaxA'] = dfA.iloc[:, -6:-1].max(axis=1) # Actual maximum, also used to calculate stack area top
dfAA['5yMax'] = dfAA['5yMaxA']-dfAA['5yMin']
#dfAA['10yAvg'] = dfA.iloc[:, -11:-1].mean(axis=1)
#dfAA['10yMin'] = dfA.iloc[:, -11:-1].min(axis=1)
#dfAA['10yMax'] = dfA.iloc[:, -11:-1].max(axis=1)
dfAA['week'] = dfA['week']

In [18]:
dfAA.head(3)

,5yAvg,5yMin,5yMaxA,5yMax,week
0,2975.2,2614,3196,582,1
1,2842.2,2533,3039,506,2
2,2704.6,2370,2947,577,3


In [19]:
dfA['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfA['week'].astype(str)+'5',format='%Y%W%w')
dfAA['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfAA['week'].astype(str)+'5',format='%Y%W%w')

In [24]:
# Create the chart
fig = px.area(dfAA, x=dfAA.weekDate, y=['5yMin', '5yMax'], #range_y=[400, 550], #range_x=[dfAA.index.min()-.2, dfAA.index.max()+.2],
              title='Stacked Area Chart with Line Chart').update_traces(fillcolor='rgba(128, 128, 128, 0.2)')

fig.update_traces(line_color='rgba(0,0,0,0)',showlegend=False)

fig.add_scatter(x=dfAA.weekDate, y=dfAA['5yAvg'], mode='lines', name='5yAvg', line=dict(color='black', width=2))
fig.add_scatter(x=dfA.weekDate, y=dfA[ly], mode='lines', name=str(ly),line=dict(color='blue', width=2))
fig.add_scatter(x=dfA.weekDate, y=dfA[ty], mode='lines+markers', name=str(ty),line=dict(color='red', width=3), marker=dict(color='red'))

# Customize the chart
fig.update_layout(autosize=False,width=1200, height=800,
    title={
        'text': '<b>Natural Gas Working Inventory</b>',
        'font': {'size': 28, 'family': 'Arial', 'color': 'black'}
        },
    xaxis_title='Week',
    yaxis_title='NG Working Inventory (Bcf)'
                  )
fig.update_traces(fillcolor='rgba(0,0,0,0)', selector=dict(name='5yMin'))

fig.add_scatter(x = [dfA['weekDate'][dfA[ty].isna().idxmax()-1]], y = [dfA[ty][dfA[ty].isna().idxmax()-1]],
                     mode = 'markers + text',
                     marker = {'color':'red', 'size':14},
                     showlegend = False,
                     text = [dfA[ty][dfA[ty].isna().idxmax()-1]],
                     textfont=dict(color='red', size=16),
                     textposition='top left') # top, bottom, middle,
fig.show()

In [26]:
dfAc = df.pivot(index='week', columns='year', values='change').astype('Int64') #.reset_index(drop='true')
dfAc = dfAc.drop(dfAc.index[-1]).reset_index()  #eliminate the add years with 53 weeks
dfAc.tail(3)

year,week,2019,2020,2021,2022,2023,2024
49,50,-107,-122,-55,-87,-87,<NA>
50,51,-161,-152,-136,-213,-87,<NA>
51,52,-58,-114,-31,-221,-14,<NA>


In [27]:
dfAAc = pd.DataFrame(dfAc.iloc[:, -6:-1].mean(axis=1))
dfAAc.columns = ['5yAvg']
dfAAc['5yMin'] = dfAc.iloc[:, -6:-1].min(axis=1) # Actual minimum, also used to calculate stack area bottom
dfAAc['5yMaxA'] = dfAc.iloc[:, -6:-1].max(axis=1) # Actual maximum, also used to calculate stack area top
dfAAc['5yMax'] = dfAAc['5yMaxA']-dfAA['5yMin']
dfAAc['week'] = dfAc['week']
dfAAc.head(3)

,5yAvg,5yMin,5yMaxA,5yMax,week
0,-86.5,-179,11,-2603,1
1,-133.0,-206,-81,-2614,2
2,-137.6,-219,-86,-2456,3


In [28]:
dfAc['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfAc['week'].astype(str)+'5',format='%Y%W%w')
dfAAc['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfAAc['week'].astype(str)+'5',format='%Y%W%w')

In [29]:
categories = dfAAc['weekDate']
lower_bounds = dfAAc['5yMin']
upper_bounds = dfAAc['5yMaxA']

In [30]:
fig = go.Figure()

for i in range(len(categories)):
    fig.add_trace(go.Bar(
        x=[categories[i]],
        y=[upper_bounds[i] - lower_bounds[i]],
        base=[lower_bounds[i]],
        marker=dict(color='lightgrey'),
        showlegend=False
        #color_discrete_sequence=['blue']
        #name=str(categories[i])
    ))

fig.add_scatter(x=dfAAc.weekDate, y=dfAAc['5yAvg'], mode='markers', name='5yAvg', line=dict(color='grey', width=2))
fig.add_scatter(x=dfAc.weekDate, y=dfAc[ly], mode='markers', name=str(ly),line=dict(color='blue', width=2))
fig.add_scatter(x=dfAc.weekDate, y=dfAc[ty], mode='lines+markers', name=str(ty),line=dict(color='red', width=3), marker=dict(color='red'))

fig.add_scatter(x = [dfAc['weekDate'][dfAc[ty].isna().idxmax()-1]], y = [dfAc[ty][dfAc[ty].isna().idxmax()-1]],
                     mode = 'markers + text',
                     marker = {'color':'red', 'size':14},
                     showlegend = False,
                     text = [dfAc[ty][dfAc[ty].isna().idxmax()-1]],
                     textfont=dict(color='red', size=16),
                     textposition='bottom right') # top, bottom, middle

fig.update_layout(autosize=False,width=1200, height=700,
    title={
        'text': '<b>Natural Gas Working Inventory Weekly Changes</b>',
        'font': {'size': 28, 'family': 'Arial', 'color': 'black'}
        },
    #xaxis_title='Week',
    yaxis_title='NG Working Inventory Weekly Change (Bcf)')

# Add source link as an annotation
fig.add_annotation(
    text="Source: EIA     https://ir.eia.gov/ngs/ngs.html",
    xref="paper", yref="paper",
    x=0.95, y=0.05,
    showarrow=False,
    font=dict(size=13)
)

fig.show()